<h1 > Offensive Language Classification </h1>


In [ ]:
#import all necessary library
import pandas as pd
import langdetect
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tqdm import tqdm


In [ ]:
#load all CSV file
train =pd.read_csv("../offence_data/train.csv")
test = pd.read_csv("../offence_data/test.csv")
valid = pd.read_csv("../offence_data/validation.csv") 
test_lebel = pd.read_csv("../offence_data/test_labels.csv") 


<h1 > just visualize train , test, validation data </h1>

In [ ]:
train.head()

In [ ]:
valid.head()

In [ ]:
test.head()

In [ ]:
test_lebel.head()

<h2 style="color: green;"> adding test label into test data and drop where lebel is NAN  </h2>

In [ ]:
test['Index'] = range(1, len(test) + 1)
test_lebel['Index'] = range(1, len(test_lebel) + 1)
test = test.merge(test_lebel[['Index', 'toxic']], on='Index', how='left')
test.drop('Index', axis=1, inplace=True)
test['toxic'] = test['toxic'].dropna()
test = test.dropna(subset=['toxic'])
test['toxic'] = test['toxic'].astype(int)
test = test.rename(columns={'content': 'feedback_text'})
print(test.tail(5))

<h1 style="color: green;"> Start EDA and Data manipulation   </h1>

In [ ]:
train.head()

In [ ]:
#Lebel destribuition chart
label_cols = ['toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry']
train[label_cols].sum().plot(kind="bar", title="Label Distribution")
plt.show()



<h1> Detect laguages present in train dataset </h1>

In [ ]:
from langdetect import detect, LangDetectException
import pandas as pd

# Sample DataFrame (replace this with your real one)
# train = pd.read_csv('your_file.csv')

# Create a list to hold detected languages
detected_languages = []

# Iterate through each feedback text
for text in train['feedback_text']:
    if pd.isna(text):
        detected_languages.append(None)
    else:
        try:
            lang = detect(text)
            detected_languages.append(lang)
        except LangDetectException:
            detected_languages.append(None)  # Or use a default like 'unknown'

# Add the language column to the DataFrame
train['lang'] = detected_languages


In [ ]:
print(train['lang'].isnull().sum())

In [ ]:
train['lang'].unique()


In [ ]:
train = train.dropna(subset=['lang'])

In [ ]:
all_words = ' '.join(train['lang'].astype(str)).lower().split()


common_words = Counter(all_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Language in training set")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()

<h1>collecting multi langual stop word for further analisys </h1>

In [ ]:

nltk.download('stopwords')

# List of language names (without language codes)
languages = [
    'english', 'spanish', 'tagalog', 'estonian', 'german', 'persian (farsi)', 'afrikaans',
    'norwegian', 'somali', 'indonesian', 'french', 'vietnamese', 'slovenian', 'turkish',
    'portuguese', 'romanian', 'swahili', 'croatian', 'danish', 'albanian', 'welsh', 'italian',
    'czech', 'swedish', 'finnish', 'dutch', 'arabic', 'polish', 'bengali', 'catalan', 'hungarian',
    'lithuanian', 'slovak', 'russian', 'hebrew', 'korean', 'chinese (simplified)', 'gujarati', 
    'tamil', 'greek', 'hindi', 'thai', 'latvian', 'macedonian', 'malayalam', 'chinese (traditional)',
    'telugu', 'marathi', 'bulgarian'
]

multi_stop_words = set()

# Loop through the list of languages and update the multi_stop_words set
for lang in languages:
    try:
        # Check if stopwords exist for the language
        if lang in stopwords.fileids():
            multi_stop_words.update(stopwords.words(lang))
        else:
            print(f"No stopwords available for: {lang}")
    except Exception as e:
        print(f"Error with language {lang}: {e}")

print(f"Collected {len(multi_stop_words)} stopwords.")



In [ ]:

# Check for Missing Values
print("Missing values:\n", train.isnull().sum())

# Plot bar chart for missing values
missing_counts = train.isnull().sum()
missing_counts = missing_counts[missing_counts >= 0]  

if not missing_counts.empty:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=missing_counts.index, y=missing_counts.values, palette="rocket")
    plt.title("Missing Values per Column")
    plt.ylabel("Number of Missing Values")
    plt.xlabel("Column Name")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Drop rows with missing feedback_text if any
missing_texts = train[train['feedback_text'].isnull()]
print(f"\nRows with missing feedback_text: {len(missing_texts)}")
if len(missing_texts) > 0:
    train = train.dropna(subset=['feedback_text'])

# Word Distribution and Sentence Length

train['word_count'] = train['feedback_text'].apply(lambda x: len(str(x).split()))
train['char_count'] = train['feedback_text'].apply(lambda x: len(str(x)))

# Plot histograms
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(train['word_count'], bins=50, ax=axes[0], color='skyblue')
axes[0].set_title('Word Count Distribution')

sns.histplot(train['char_count'], bins=50, ax=axes[1], color='salmon')
axes[1].set_title('Character Count Distribution')
plt.tight_layout()
plt.show()

# Most Common Words (Before Preprocessing)


# Get the list of English stopwords
# Remove stop words from feedback_text
all_words = ' '.join(train['feedback_text'].astype(str)).lower().split()
filtered_words = [word for word in all_words if word not in multi_stop_words]

# Count the most common words
common_words = Counter(filtered_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Words (after removing multi langual  Stopwords only)")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()


In [ ]:


# Function to clean the text (remove stopwords, punctuation, numbers, etc.)
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation using str.translate
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers and special characters using regular expressions
    text = re.sub(r'\d+', '', text)

    # Remove stopwords
    words = text.split()
    cleaned_words = [word for word in words if word not in multi_stop_words ]

    return ' '.join(cleaned_words)

# Apply the clean_text function to the feedback_text column
train['cleaned_feedback_text'] = train['feedback_text'].apply(lambda x: clean_text(str(x)))

# Combine all cleaned words
all_words = ' '.join(train['cleaned_feedback_text']).split()

# Count the most common words
common_words = Counter(all_words)
common_words_df = pd.DataFrame(common_words.most_common(20), columns=['word', 'count'])

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=common_words_df, x='count', y='word', palette='mako')
plt.title("Top 20 Most Common Words (After removing stopword and other text noise)")
plt.xlabel("Count")
plt.ylabel("Word")
plt.show()


<h1 style="color: green;"> Word Frequency Map  </h1>

In [ ]:
from wordcloud import WordCloud

# Combine all feedback_text into one string
all_text = ' '.join(train['feedback_text'].astype(str)).lower()

# Generate Word Cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=multi_stop_words,
    max_words=200,
    colormap='plasma'
).generate(all_text)

# Plot Word Cloud
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("🔤 Word Frequency Map (Word Cloud)", fontsize=20)
plt.show()


<h1> balancing the  dataset </h1>

In [ ]:
#badly needed but donot get time to do it 

<h1>preprocessing , training , evaluation <h1>

In [ ]:


# Setup
warnings.filterwarnings('ignore')
nltk.download('wordnet')

# Load Dataset
train_df = train
test_df = test  
# Preprocessing
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'@\w+|\#', '', text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in multi_stop_words]
    return " ".join(tokens)

train_df['clean_text'] = train_df['feedback_text'].apply(preprocess)
test_df['clean_text'] = test_df['feedback_text'].apply(preprocess)

# Custom Transformer for Metadata
class MetadataExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns].values


# Features & Labels

text_features = train_df['clean_text']
text_test_features = test_df['clean_text']

meta_features = ['abusive', 'vulgar', 'menace', 'offense', 'bigotry']
train_df[meta_features] = train_df[meta_features].fillna(0)

# Fill missing with 0 in test if any of these columns are missing
for col in meta_features:
    if col not in test_df.columns:
        test_df[col] = 0

X = train_df[['clean_text'] + meta_features]
y = train_df['toxic']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('tfidf', TfidfVectorizer(max_features=10000), 'clean_text'),
    ('meta', StandardScaler(), meta_features)
])

pipeline = Pipeline([
    ('features', preprocessor),
    ('clf', LogisticRegression(solver='liblinear'))
])


# Train Model

pipeline.fit(X_train, y_train)


# Evaluate on Validation Set

y_pred = pipeline.predict(X_val)
y_pred_proba = pipeline.predict_proba(X_val)[:, 1]

print("\n📊 Evaluation on Validation Set (toxic only):")
print(f"Accuracy: {accuracy_score(y_val, y_pred):.4f}")
print(classification_report(y_val, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_val, y_pred_proba):.4f}")

# Confusion Matrix
sns.heatmap(confusion_matrix(y_val, y_pred), annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - Toxic")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# AUC-ROC curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Test Set (Toxic)')
plt.legend(loc='lower right')
plt.grid()
plt.show()



In [ ]:

# Predict and evaluation on Test Set

Y_test = test_df['toxic']
X_test = test_df[['clean_text'] + meta_features]
test_preds = pipeline.predict(X_test)
test_proba = pipeline.predict_proba(X_test)[:, 1]

# ✅ Evaluate on Test Set (if it has 'toxic' labels)
if 'toxic' in test_df.columns:
    print("\n📊 Evaluation on Test Set (toxic only):")
    print(f"Accuracy: {accuracy_score(Y_test, test_preds):.4f}")
    print(classification_report(Y_test, test_preds))
    print(f"AUC-ROC: {roc_auc_score(Y_test, test_proba):.4f}")
    
    # Confusion Matrix for Test Set
    sns.heatmap(confusion_matrix(Y_test, test_preds), annot=True, fmt="d", cmap="Greens")
    plt.title("Confusion Matrix - Test Set (Toxic)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()




In [ ]:

# ROC Curve for test set (on test data)
fpr, tpr, thresholds = roc_curve(test_df['toxic'], test_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Test Set (Toxic)')
plt.legend(loc='lower right')
plt.grid()
plt.show()


<h1> LSTM implementation </h1>

In [ ]:

# Preprocessing
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'@\w+|\#', '', text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in multi_stop_words]
    return " ".join(tokens)

def simple_tokenizer(text):
    return text.split()

def build_vocab(texts, min_freq=2):
    counter = Counter()
    for text in texts:
        tokens = simple_tokenizer(text)
        counter.update(tokens)
    vocab = {"<unk>": 0}
    for token, freq in counter.items():
        if freq >= min_freq:
            vocab[token] = len(vocab)
    return vocab

def encode_tokens(tokens, vocab, max_len=100):
    token_ids = [vocab.get(token, 0) for token in tokens]
    return token_ids[:max_len] + [0]*(max_len - len(token_ids[:max_len]))

# Dataset Class
class CustomLSTMDataset(Dataset):
    def __init__(self, texts, labels=None, meta=None, vocab=None, max_len=100):
        self.texts = [encode_tokens(simple_tokenizer(t), vocab, max_len) for t in texts]
        self.labels = labels
        self.meta = meta

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.texts[idx], dtype=torch.long)
        }
        if self.meta is not None:
            item['meta'] = torch.tensor(self.meta[idx], dtype=torch.float)
        if self.labels is not None:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
# Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, meta_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim + meta_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, meta):
        embeds = self.embedding(input_ids)
        _, (hidden, _) = self.lstm(embeds)
        combined = torch.cat((hidden[-1], meta), dim=1)
        out = self.fc(combined)
        return self.sigmoid(out)
# Load and preprocess train data
train_df = train
train_df['clean_text'] = train_df['feedback_text'].apply(preprocess)

X = train_df['clean_text']
y = train_df['toxic'].values
meta = train_df[['abusive', 'vulgar', 'menace', 'offense', 'bigotry']].values
scaler = StandardScaler()
meta_scaled = scaler.fit_transform(meta)

X_train, X_val, y_train, y_val, meta_train, meta_val = train_test_split(
    X, y, meta_scaled, test_size=0.2, random_state=42
)

vocab = build_vocab(X_train)

train_dataset = CustomLSTMDataset(X_train, y_train, meta_train, vocab)
val_dataset = CustomLSTMDataset(X_val, y_val, meta_val, vocab)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


# Train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(len(vocab), 100, 128, meta_dim=5, output_dim=1).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(123):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        meta = batch['meta'].to(device)
        labels = batch['label'].unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, meta)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


# Evaluate

model.eval()
preds, probs, true = [], [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        meta = batch['meta'].to(device)
        labels = batch['label'].unsqueeze(1).to(device)
        outputs = model(input_ids, meta)
        probs.extend(outputs.cpu().numpy())
        preds.extend((outputs > 0.5).cpu().numpy())
        true.extend(labels.cpu().numpy())

true = np.array(true)
preds = np.array(preds)
probs = np.array(probs)

print("\nClassification Report (Validation - Toxic Only):")
print(classification_report(true, preds))
print("Accuracy:", accuracy_score(true, preds))
print("Precision:", precision_score(true, preds))
print("Recall:", recall_score(true, preds))
print("F1 Score:", f1_score(true, preds))
print("ROC AUC:", roc_auc_score(true, probs))
print("Confusion Matrix:\n", confusion_matrix(true, preds))
 
# Evaluate on Validation Set graph chart


print("\n📊 Evaluation on Validation Set (toxic only):")
print(f"Accuracy: {accuracy_score(true, preds):.4f}")
print(classification_report(true, preds))
print(f"AUC-ROC: {roc_auc_score(true, preds):.4f}")

# Confusion Matrix
sns.heatmap(confusion_matrix(true, preds), annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - Toxic")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# AUC-ROC curve
fpr, tpr, thresholds = roc_curve(true, probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Test Set (Toxic)')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Test Prediction (toxic only)

test_df = test
test_df['clean_text'] = test_df['feedback_text'].apply(preprocess)
test_meta = scaler.transform(test_df[['abusive', 'vulgar', 'menace', 'offense', 'bigotry']].values)

test_dataset = CustomLSTMDataset(test_df['clean_text'], meta=test_meta, vocab=vocab)
test_loader = DataLoader(test_dataset, batch_size=32)

model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        meta = batch['meta'].to(device)
        outputs = model(input_ids, meta)
        predictions = (outputs > 0.5).int().cpu().numpy()
        test_preds.extend(predictions.flatten())


In [ ]:
# Predict and evaluation on Test Set

Y_test = test_df['toxic']
model.eval()
T_preds, T_probs, T_true = [], [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        meta = batch['meta'].to(device)
        
        outputs = model(input_ids, meta)
        T_probs.extend(outputs.cpu().numpy())
        T_preds.extend((outputs > 0.5).cpu().numpy())
        T_true.extend(labels.cpu().numpy())

T_true = np.array(T_true)
T_preds = np.array(T_preds)
T_probs = np.array(T_probs)


# ✅ Evaluate on Test Set (if it has 'toxic' labels)
if 'toxic' in test_df.columns:
    print("\n📊 Evaluation on Test Set (toxic only):")
    print(f"Accuracy: {accuracy_score(Y_test, T_preds):.4f}")
    print(classification_report(Y_test, T_preds))
    print(f"AUC-ROC: {roc_auc_score(Y_test, T_probs):.4f}")
    
    # Confusion Matrix for Test Set
    sns.heatmap(confusion_matrix(Y_test, T_preds), annot=True, fmt="d", cmap="Greens")
    plt.title("Confusion Matrix - Test Set (Toxic)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [ ]:

# ROC Curve for test set (on test data)
fpr, tpr, thresholds = roc_curve(test_df['toxic'], T_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Test Set (Toxic)')
plt.legend(loc='lower right')
plt.grid()
plt.show()